In [2]:
from mmdet3d.core.bbox.iou_calculators import *
import torch

In [60]:
bbox1 = torch.tensor([1, 2, 3, 400, 400, 400, 1])
bbox2 = torch.tensor([[1, 2, 3, 200, 200, 600, 0.5], [1, 2, 3, 200, 200, 600, 0.5]])
bbox_overlaps_3d(bbox1.unsqueeze(0), bbox2, coordinate='lidar')

tensor([[0.2222, 0.2222]])

In [59]:
# method 2
import numpy as np
import cv2
from shapely.geometry import Polygon

# y is pointing upwards
def boxes_3d_to_corners(boxes_3d):
    all_corners = []
    for box_3d in boxes_3d:
        x3d, y3d, z3d, l, h, w, yaw = box_3d
        R = np.array([[np.cos(yaw),  0,  np.sin(yaw)],
                      [0,            1,  0          ],
                      [-np.sin(yaw), 0,  np.cos(yaw)]]);
        corners = np.array([[ l/2,  0.0,  w/2],  # front up right
                            [ l/2,  0.0, -w/2],  # front up left
                            [-l/2,  0.0, -w/2],  # back up left
                            [-l/2,  0.0,  w/2],  # back up right
                            [ l/2, -h,  w/2],  # front down right
                            [ l/2, -h, -w/2],  # front down left
                            [-l/2, -h, -w/2],  # back down left
                            [-l/2, -h,  w/2]]) # back down right
        r_corners = corners.dot(np.transpose(R))
        cam_points_xyz = r_corners+np.array([x3d, y3d, z3d])
        all_corners.append(cam_points_xyz)
    return np.array(all_corners)

def overlapped_boxes_3d(single_box, box_list):
    x0_max, y0_max, z0_max = np.max(single_box, axis=0)
    x0_min, y0_min, z0_min = np.min(single_box, axis=0)
    overlap = np.zeros(len(box_list))
    for i, box in enumerate(box_list):
        x_max, y_max, z_max = np.max(box, axis=0)
        x_min, y_min, z_min = np.min(box, axis=0)
        if x0_max < x_min or x0_min > x_max:
            overlap[i] = 0
            continue
        if y0_max < y_min or y0_min > y_max:
            overlap[i] = 0
            continue
        if z0_max < z_min or z0_min > z_max:
            overlap[i] = 0
            continue
        x_draw_min = min(x0_min, x_min)
        x_draw_max = max(x0_max, x_max)
        z_draw_min = min(z0_min, z_min)
        z_draw_max = max(z0_max, z_max)
        offset = np.array([x_draw_min, z_draw_min])
        buf1 = np.zeros((int(z_draw_max-z_draw_min), int(x_draw_max-x_draw_min)),
            dtype=np.int32)
        buf2 = np.zeros_like(buf1)
        cv2.fillPoly(buf1, np.int32([single_box[:4, [0,2]]-offset]), color=1)
        cv2.fillPoly(buf2, np.int32([box[:4, [0,2]]-offset]), color=1)
        shared_area = cv2.countNonZero(buf1*buf2)
        area1 = cv2.countNonZero(buf1)
        area2 = cv2.countNonZero(buf2)
        shared_y = min(y_max, y0_max) - max(y_min, y0_min)
        intersection = shared_y * shared_area
        union = (y_max-y_min) * area2 + (y0_max-y0_min) * area1
        overlap[i] = np.float32(intersection) / (union - intersection)
    return overlap

def overlapped_boxes_3d_fast_poly(single_box, box_list):
    single_box_max_corner = np.max(single_box, axis=0)
    single_box_min_corner = np.min(single_box, axis=0)
    x0_max, y0_max, z0_max = single_box_max_corner
    x0_min, y0_min, z0_min = single_box_min_corner
    max_corner = np.max(box_list, axis=1)
    min_corner = np.min(box_list, axis=1)
    overlap = np.zeros(len(box_list))
    non_overlap_mask =  np.logical_or(single_box_max_corner < min_corner,
        single_box_min_corner > max_corner)
    non_overlap_mask = np.any(non_overlap_mask, axis=1)
    p1  = Polygon(single_box[:4, [0,2]])
    area1 = p1.area
    for i in range(len(box_list)):
        if not non_overlap_mask[i]:
            x_max, y_max, z_max = max_corner[i]
            x_min, y_min, z_min = min_corner[i]
            p2 =  Polygon(box_list[i][:4, [0,2]])
            shared_area = p1.intersection(p2).area
            area2 = p2.area
            shared_y = min(y_max, y0_max) - max(y_min, y0_min)
            intersection = shared_y * shared_area
            union = (y_max-y_min) * area2 + (y0_max-y0_min) * area1
            overlap[i] = np.float32(intersection) / (union - intersection)
    return overlap

bbox1_corner = boxes_3d_to_corners([bbox1])
bbox2_corner = boxes_3d_to_corners([bbox2])
overlapped_boxes_3d_fast_poly(bbox1_corner[0], bbox2_corner)

array([0.22501061])

In [43]:
buf = np.zeros((4,4))
cv2.fillPoly(buf, np.array([[[3, 3], [3, 1], [1, 1], [1, 3]]]), color=1)

array([[0., 0., 0., 0.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]])